# Automatic Differentiation with torch.autograd
# 自动求导

In [2]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [4]:
print(f"z的梯度函数 Gradient function for z = {z.grad_fn}")
print(f"loss的梯度函数 Gradient function for loss = {loss.grad_fn}")

z的梯度函数 Gradient function for z = <AddBackward0 object at 0x00000274E5EF7610>
loss的梯度函数 Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000002749E05FC40>


In [5]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3124, 0.1760, 0.0919],
        [0.3124, 0.1760, 0.0919],
        [0.3124, 0.1760, 0.0919],
        [0.3124, 0.1760, 0.0919],
        [0.3124, 0.1760, 0.0919]])
tensor([0.3124, 0.1760, 0.0919])


## Disabling Gradient Tracking
## 禁用渐变跟踪

默认情况下，所有 的 requires_grad=True 张量都在跟踪其计算历史并支持梯度计算。但是，在某些情况下我们不需要这样做，例如，当我们训练了模型并只想将其应用于一些输入数据时，即我们只想通过网络进行前向计算。我们可以通过将计算代码括起来用 torch.no_grad() block 来停止跟踪计算：

In [6]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [7]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


There are reasons you might want to disable gradient tracking:

出于多种原因，您可能希望禁用渐变跟踪：

To mark some parameters in your neural network as frozen parameters.

将神经网络中的某些参数标记为冻结参数。

To speed up computations when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.

在仅执行前向传递时加快计算速度，因为在不跟踪梯度的张量上进行计算会更高效。

## More on Computational Graphs
## 有关计算图的更多信息
Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of Function objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

从概念上讲，autograd 在由 Function 对象组成的有向无环图 （DAG） 中保存数据（张量）和所有已执行作（以及生成的新张量）的记录。在这个 DAG 中，叶子是输入张量，根是输出张量。通过从根到叶跟踪此图，您可以使用链式法则自动计算梯度。

In a forward pass, autograd does two things simultaneously:

在前向传递中，autograd 同时执行两件事：

run the requested operation to compute a resulting tensor

运行请求的作以计算生成的 Tensor

maintain the operation’s gradient function in the DAG.

在 DAG 中保持作的 gradient 函数。

The backward pass kicks off when .backward() is called on the DAG root. autograd then:

在 DAG 根上调用时 .backward() ，将启动向后传递。 autograd 然后：

computes the gradients from each .grad_fn,

计算每个 .grad_fn 的梯度 ，

accumulates them in the respective tensor’s .grad attribute

将它们累积到相应 Tensor 的属性中 .grad

using the chain rule, propagates all the way to the leaf tensors.

使用 chain rule，一直传播到 Leaf Tensors。